In [1]:
import tensorflow as tf

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

def pipe(data, batch_size = 128, shuffle = False):
    dataset = tf.data.Dataset.from_tensor_slices(data)
    if shuffle:
        dataset = dataset.shuffle(buffer_size = batch_size * 10)
    dataset = dataset.batch(batch_size)
    #dataset = dataset.prefetch((batch_size * 2) + 1)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

(tr_x, tr_y), (te_x, te_y) = tf.keras.datasets.cifar10.load_data()

tr_x = tr_x * 1/255
te_x = te_x * 1/255

batch_size = 128

tr_data = pipe((tr_x, tr_y), batch_size = batch_size, shuffle = True)
te_data = pipe((te_x, te_y), batch_size = batch_size, shuffle = False)

170508288/170498071 [==============================] - 13s 0us/step


In [2]:
def get_model(shape = (32, 32, 3), weights = "imagenet"):
    model = tf.keras.applications.ResNet50(input_shape = shape, include_top = False, weights = weights)
    flatten = tf.keras.layers.GlobalAveragePooling2D()(model.output)
    drop_out = tf.keras.layers.Dropout(0.5)(flatten)
    dense = tf.keras.layers.Dense(2048, activation = "relu")(drop_out)
    prediction = tf.keras.layers.Dense(10, activation = "softmax", name = "prediction")(dense)
    model = tf.keras.Model(model.input, prediction)
    return model

result = {}
for index in range(5):
    print("{0} - model learn".format(index + 1))
    model = get_model()
    loss = tf.keras.losses.sparse_categorical_crossentropy
    opt = tf.keras.optimizers.Adam(1e-4)
    metric = [tf.keras.metrics.sparse_categorical_accuracy]
    model.compile(loss = loss, optimizer = opt, metrics = metric)
    model.fit(tr_data, epochs = 10, verbose = 0)

    loss, score = model.evaluate(te_data, verbose = 1)
    path = "model{0}.h5".format(index + 1)

    result[path] = score
    model.save_weights(path)
print("finish")

1 - model learn
79/79 [==============================] - 3s 19ms/step - loss: 1.0081 - sparse_categorical_accuracy: 0.7860
2 - model learn
79/79 [==============================] - 3s 21ms/step - loss: 0.9853 - sparse_categorical_accuracy: 0.7909
3 - model learn
79/79 [==============================] - 3s 20ms/step - loss: 0.9366 - sparse_categorical_accuracy: 0.7930
4 - model learn
79/79 [==============================] - 3s 19ms/step - loss: 0.9480 - sparse_categorical_accuracy: 0.7942
5 - model learn
79/79 [==============================] - 3s 19ms/step - loss: 0.9691 - sparse_categorical_accuracy: 0.7995
finish


In [3]:
import os
import numpy as np
import model_soup

print("[Original Performance]")
for k, v in result.items():
    print("[{0}] accuracy:{1:.4f}".format(os.path.basename(k), v))

print("\n[Greedy Soup Performance]")
greedy_model = model_soup.tf.greedy_soup(model, list(result.keys()), te_data, metric = tf.keras.metrics.sparse_categorical_accuracy, compare = np.greater_equal)
loss, score = greedy_model.evaluate(te_data)

print("\n[Uniform Soup Performance]")
uniform_model = model_soup.tf.uniform_soup(model, list(result.keys()))
loss, score = uniform_model.evaluate(te_data)

[Original Performance]
[model1.h5] accuracy:0.786000
[model2.h5] accuracy:0.790900
[model3.h5] accuracy:0.793000
[model4.h5] accuracy:0.794200
[model5.h5] accuracy:0.799500

[Greedy Soup Performance]
[model1.h5] step: 79 - time: 7.62s - sparse_categorical_accuracy: 0.7860
[model2.h5] step: 79 - time: 7.48s - sparse_categorical_accuracy: 0.8077
[model3.h5] step: 79 - time: 7.74s - sparse_categorical_accuracy: 0.8124
[model4.h5] step: 79 - time: 7.44s - sparse_categorical_accuracy: 0.8088
[model5.h5] step: 79 - time: 7.89s - sparse_categorical_accuracy: 0.8153
model soup best score : 0.8152999877929688
79/79 [==============================] - 2s 20ms/step - loss: 0.8123 - sparse_categorical_accuracy: 0.8153

[Uniform Soup Performance]
79/79 [==============================] - 2s 21ms/step - loss: 1.3808 - sparse_categorical_accuracy: 0.8114
